# ImageCropper Class
Takes folder of full screenshots and crops them (top and bottom) and saves to specified folders.

In [50]:
import os
import pandas as pd
import numpy as np
import cv2

class ImageCropper:

    def __init__(self, root_dir='image_data', kind='test'):
        self.x = 0
        self.y = 25
        self.h = 140 - self.y
        self.w = 1280
        self.play_width = 347
        self.play_height = 206
        self.play_loc_r = (423, 821)
        self.play_loc_l = (423, 94)
        self.root_dir = root_dir
        self.kind_path = os.path.join(root_dir, kind)
        self.user_path = os.path.join(root_dir, kind, 'users')
        self.play_path = os.path.join(root_dir, kind, 'saved_plays')
        self.check_for_folders()
    
    def check_for_folders(self):
        for path in [self.root_dir, self.kind_path, self.user_path, self.play_path]:
            if not os.path.isdir(path):
                os.mkdir(path)
    
    def crop_image_top(self, img_name):
        # Open Image
        image = cv2.imread(img_name)
        # Crop Image
        crop_img = image[self.y:self.y + self.h, :]

        left = crop_img[:, 72:262]
        right = crop_img[:, 1015:1208]

        # Combine
        concat_img = np.concatenate([left, right], axis=1)
        fn = self.save_file(concat_img, img_name.split('/')[-1], self.user_path, 'top')
        return fn


    def crop_image_play(self, img_name):
        # Open Image
        image = cv2.imread(img_name)
        # Crop Image
        left = image[self.play_loc_l[0]: self.play_loc_l[0] + self.play_height,
                     self.play_loc_l[1]: self.play_loc_l[1] + self.play_width]
        right = image[self.play_loc_r[0]: self.play_loc_r[0] + self.play_height,
                     self.play_loc_r[1]: self.play_loc_r[1] + self.play_width]
        # Combine
        concat_img = np.concatenate([left, right], axis=1)  #
        fn = self.save_file(concat_img, img_name.split('/')[-1], self.play_path, 'bottom')
        return fn


    def save_file(self, item, filename, save_path, kind):
        filename = filename.lstrip('saved_frames/')
        save_filename = f'{save_path}/{filename.rstrip(".jpg")}-{kind}.jpg'
        cv2.imwrite(save_filename, item)
        return save_filename

## Using ImageCropper

In [51]:
cropper = ImageCropper()

test_dir = 'image_data/test/raw_caps'
files = os.listdir(test_dir)
files = [os.path.join(test_dir, f) for f in files if f.endswith('.jpg')]

In [52]:
cropped_plays = [cropper.crop_image_play(f) for f in files]

In [53]:
cropped_tops = [cropper.crop_image_top(f) for f in files]

In [67]:
def construct_test_dataset(test_dir):
    filenames = []
    for root, dirs, files in os.walk(test_dir, topdown=True):
            for name in files:
                filenames.append(os.path.join(root, name))


    filenames = [f for f in filenames if 'DS_Store' not in f]
    df = pd.DataFrame({'filename': filenames})
    df.reset_index(drop=True, inplace=True)

    fn = f'csv/test_data_{test_dir.split("/")[-1]}.csv'
    df.to_csv(fn)
    return fn

In [70]:
side_dir = 'image_data/test/saved_plays'
fn = construct_test_dataset(side_dir)

In [71]:
df = pd.read_csv(fn, index_col=0)
df.head()

,filename
0,image_data/test/saved_plays/Madden NFL 19_2019...
1,image_data/test/saved_plays/Madden NFL 19_2018...
2,image_data/test/saved_plays/Madden NFL 19_2019...
3,image_data/test/saved_plays/Madden NFL 19_2019...
4,image_data/test/saved_plays/Madden NFL 19_2019...
